#### Functions (IGNORE)

In [5]:
import yfinance as yf
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [6]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [7]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

In [13]:
import random

def stochastic_modeling(nested_dict, tickers, periods,num_samples):
    # Store the returns in a nested dictionary
    nested_dict_returns = {period: {ticker: [] for ticker in tickers} for period in periods}

    # Go through each economic time period
    for period in periods:
        max_index = len(nested_dict[period]) - 30  # Ensure there's enough data to calculate ROI

        # Generate random samples from the valid range
        random_dates = random.choices(range(max_index), k=num_samples)

        for ticker in tickers:
            for date_idx in random_dates:
                start_price = nested_dict[period][ticker].iloc[date_idx]
                end_price = nested_dict[period][ticker].iloc[date_idx + 30]

                # Get the return by the Holding Period Return
                roi = (((end_price - start_price) / start_price) * 100)

                nested_dict_returns[period][ticker].append(roi)

    return nested_dict_returns  # Return the nested dictionary with returns

In [24]:
def stochastic_roi(tickers,periods,return_rates_list,analysis_type):
    df = pd.DataFrame(index=tickers,columns=periods)
    for period in periods:
        for ticker in tickers:
            data = pd.Series(return_rates_list[period][ticker])
            if analysis_type=='Mean':
                df.at[ticker,period] = data.mean()
            elif analysis_type=='Median':
                df.at[ticker,period] = data.median()
            elif analysis_type=='Std':
                df.at[ticker,period] = data.std()
            elif analysis_type=='Variance':
                df.at[ticker,period] = data.var()

    return df

## Buy and Hold Investment Technique
The buy and hold strategy is a passive investing strategy that will be applied to the 11 sector ETFs during different macroeconomic time periods.

In [8]:
# import packages
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction']

In [10]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

In [11]:
# save nested dictionary data as a variable to be accessed.
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [16]:
# get adjusted close price
sector_etf_adjusted_close = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

### Perform stochastic modeling using buy and hold strategy
Use a different day where the stock begins investing then hold for a month and see the return.

In [26]:
# perform stochastic modeling on the buy and 
stochastic_buy_hold = stochastic_modeling(sector_etf_adjusted_close,sector_etf_tickers,economic_cycle_periods_list,10000)

In [27]:
stochastic_roi(sector_etf_tickers,economic_cycle_periods_list,stochastic_buy_hold,'Mean')

,trough,expansion,peak,contraction
XLB,1.272944,1.48503,1.198677,-1.186616
XLI,-0.873409,2.164375,2.022506,-1.944523
XLF,-2.204338,2.637947,2.766903,-4.647609
XLK,1.529493,2.111732,4.472099,-2.649538
XLY,2.231488,2.540581,1.447289,-0.954601
XLP,-1.393381,2.109823,1.949633,-0.079942
XLE,0.26154,0.794221,-0.098585,-1.083695
XLV,-0.969988,3.070623,2.772104,-1.649997
VOX,2.578617,1.75983,2.452495,-3.364605
XLU,-1.14542,1.773651,2.334167,-2.595899


## Bollinger Bands Investment Technique
Using John Bollinger's techniques 'Bollinger Bands' to create buy and sell signals to observe the roi for investing for a month.

In [30]:
# add bollinger data
import scipy.stats as stats
def add_bollinger_data(data,window,conf_int):
        z_score = stats.norm.ppf(1 - (1 - conf_int) / 2) # create a zscore from the mean

        data['middle_band'] = data['Adj Close'].rolling(window).mean()
        data['upper_band'] = data['middle_band'] + z_score * data['Adj Close'].rolling(window).std()
        data['lower_band'] = data['middle_band'] - z_score * data['Adj Close'].rolling(window).std()

        data['Signal'] = None

        data['Signal'] = np.where(data['Adj Close'] < data['lower_band'], 'Buy', 
                              np.where(data['Adj Close'] > data['upper_band'], 'Sell', np.nan))

        return data

In [33]:
# create bollinger data for multiple time period and multiple tickers
def bollinger_data_multiple_periods_tickers(periods,tickers,data,window,confidence_period):
    # for each ticker in economic time periods
    for period in periods:
            for ticker in tickers:
                    try:
                        add_bollinger_data(data[period][ticker],window,confidence_period)
                    except KeyError:
                        print(f'Data for {ticker} does not exist during {period}')

In [35]:
# create bollinger bands in stock data
bollinger_data_multiple_periods_tickers(economic_cycle_periods_list,sector_etf_tickers,sector_etf_data,20,0.95)

In [52]:
# example case of bollinger bands in stock data
sector_etf_data['expansion']['XLB']['Signal'].value_counts()

nan     670
Buy      45
Sell     39
Name: Signal, dtype: int64

In [60]:
def collect_signals(nested_dict, periods, tickers):
    # Initialize an empty dictionary to hold DataFrames for each period
    bb_nested_dict = {}

    for period in periods:
        # Create a DataFrame for each period with the tickers as columns
        signals_period = pd.DataFrame(columns=tickers)
        
        # Loop through each ticker and extract the 'Signal'
        for ticker in tickers:
            signals_period[ticker] = nested_dict[period][ticker]['Signal']
        
        # Store the DataFrame in the dictionary using the period as the key
        bb_nested_dict[period] = signals_period

    # Return the dictionary containing DataFrames for each period
    return bb_nested_dict

In [68]:
bb_bands_signals = collect_signals(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)

In [72]:
len(sector_etf_adjusted_close) == len(bb_bands_signals)

True

In [ ]:
# create function for signals 
def bb_band_roi(bb_signals_nd, adj_close_nd,periods,tickers,n_sample):
    # create a nested dictionary with technical analysis signals and adj close price as pages
    all_data = {
        'Adj Close': adj_close_nd,
        'Bollinger Band': bb_signals_nd
    }

    # create start index dates
    random_dates = random.choices(range(max_index),k=n_sample)

    # go through each period
    for period in periods:
        # go through each day in the signals then collect the location
        for row_idx, row in all_data['Bollinger Band'][period].iterrows():
            for col_idx,value in enumerate(row):
                for ticker in tickers:
                    if row[value]=='Buy':
                        adj_close_price = adj_close_nd.iloc[col_idx,row_idx]
                        print(f'buy ticker {col_idx} at day {row_idx} ')
                        print(f'at price {adj_close_price}')
                    elif row[ticker] == 'Sell':
                        print(f'sell ticker {col_idx} at day {row_idx}')
                        adj_close_price = adj_close_nd.iloc[col_idx,row_idx]
                        print(f'at price {adj_close_price}')
                    else: continue

        


    # get the max index so that there is a month of investing time
    max_index = len(adj_close_nd[period]-30)

    # this is your own keep working

        

In [75]:
import random

def bb_band_roi(bb_signals_nd, adj_close_nd, periods, tickers, n_sample):
    # Ensure we have valid data and calculate max index
    if adj_close_nd is None or len(adj_close_nd) < 30:
        raise ValueError("Not enough data for a 30-day investment period")

    # Get the max index so that there is a month (30 days) of investing time
    max_index = len(adj_close_nd) - 30

    # Generate random start indices for sampling investment periods
    random_dates = random.choices(range(max_index), k=n_sample)

    # Create a nested dictionary with technical analysis signals and adj close price as pages
    all_data = {
        'Adj Close': adj_close_nd,
        'Bollinger Band': bb_signals_nd
    }

    # Loop through each period
    for period in periods:
        # Loop through each sampled starting date (randomized investment windows)
        for start_idx in random_dates:
            end_idx = start_idx + 30  # Define the investment period to last for 30 days

            # Loop through each day within this 30-day period
            for row_idx, row in all_data['Bollinger Band'][period].iloc[start_idx:end_idx].iterrows():
                # Loop through each ticker
                for col_idx, ticker in enumerate(tickers):
                    signal = row[ticker]  # Get the Buy/Sell signal for the ticker
                    adj_close_price = all_data['Adj Close'].iloc[row_idx, col_idx]

                    # Execute trade based on signal
                    if signal == 'Buy':
                        print(f"Buy ticker {ticker} at day {row_idx}, price {adj_close_price}")
                    elif signal == 'Sell':
                        print(f"Sell ticker {ticker} at day {row_idx}, price {adj_close_price}")

                    # Continue to next day if no signal is generated
                    else:
                        continue

In [76]:
bb_band_roi(bb_bands_signals,sector_etf_adjusted_close,economic_cycle_periods_list,sector_etf_tickers,10)

ValueError: Not enough data for a 30-day investment period